In [39]:
import sys, pygame, random
from math import *
from pygame import *
import time

In [40]:
# Pygame parameters
windowSize = [640, 480]
nodeStep = 10.0
pygame.init()
fpsClock = pygame.time.Clock()
screen = pygame.display.set_mode(windowSize)

# Defining colour RGB scales for simulation window
white = 255, 255, 255
black = 0, 0, 0
red = 255, 0, 0
blue = 0, 255, 0
green = 0, 0, 255
cyan = 0,100,78
count = 0

# Initializing the obstacle array
rectObs = []

In [42]:
# Defining the various functions for RRT implementation
class treeNode(object):
    def __init__(self, point, parent):
        super(treeNode, self).__init__()
        self.point = point
        self.parent = parent


#distance between two points
def calcDistance(x,y):
    return sqrt((x[0]-y[0])*(x[0]-y[0])+(x[1]-y[1])*(x[1]-y[1]))

#check if the node is within certain distance from the start and goal point circles
def checkCollision(x, y, radius):
    distance = calcDistance(x,y)
    if (distance <= radius):
        return True
    return False

#compute distance of the node from the parent node
def nodesDistance(x,y):
    if calcDistance(x,y) < nodeStep:
        return y
    else:
        theta = atan2(y[1]-x[1],y[0]-x[0])        
        return x[0] + nodeStep*cos(theta), x[1] + nodeStep*sin(theta)

# Check if the point collides with the obstacle
def collideObstacle(p): 
    for rect in obstacles:
        if rect.collidepoint(p) == True:
            return True
    return False


# Randomly guessing a node position over the robot workspace
# This includes a random angle of 0 to 360, instead of restricting it to only 180
def randomNode():
    while True:
        p = random.random()*640, random.random()*480
        noCollision = collideObstacle(p)
        if noCollision == False:
            return p

# Adding the obstacles and defining the workspace
def init_workspace():  
    global obstacles
    obstacles = []

    # top & left wall
    obstacles.append(pygame.Rect((0,0),(640,15)))
    obstacles.append(pygame.Rect((0,0),(15,480)))

    # bottom & right wall
    obstacles.append(pygame.Rect((0,465),(640,15)))
    obstacles.append(pygame.Rect((625,0),(15,480)))

    # left rectangle
    obstacles.append(pygame.Rect((0,200),(450,100)))

    # right rectangle
    obstacles.append(pygame.Rect((500,200),(140,100)))

    for obstacle in obstacles:
        pygame.draw.rect(screen, black, obstacle)


def reset():
    global totalNodes
    screen.fill(white)
    init_workspace()
    totalNodes = 0


def startRRT():
    
    global totalNodes
    
    # initialing various parameters for RRT
    startPointSet = False
    startPoint = treeNode(None, None)
    goalPoseSet = False
    goalPoint = treeNode(None, None)
    currentState='init'
    nodes = []
    reset()

    while True:
        if currentState == 'init':
            print('goal point not yet set')
            fpsClock.tick(10)

        # constructing the final path from the start to the goal point    
        elif currentState == 'goalFound':
            currentNode = goalNode.parent
            print("Goal Reached")
            time.sleep(3)
            while currentNode.parent != None:
                pygame.draw.line(screen,red,currentNode.point,currentNode.parent.point)
                currentNode = currentNode.parent

        elif currentState == 'optimize':
            fpsClock.tick(0.5)
            pass
        elif currentState == 'buildTree':

            # Keeping a total count of the nodes and restricting it to 5000
            totalNodes = totalNodes+1
            if totalNodes < 5000:
                foundNext = False
                while foundNext == False:
                    randPoint = randomNode()
                    parentNode = nodes[0]
                    for p in nodes:
                        if calcDistance(p.point,randPoint) <= calcDistance(parentNode.point,randPoint):
                            newPoint = nodesDistance(p.point,randPoint)
                            if collideObstacle(newPoint) == False:
                                parentNode = p
                                foundNext = True

                newnode = nodesDistance(parentNode.point,randPoint)
                nodes.append(treeNode(newnode, parentNode))
                pygame.draw.line(screen,cyan,parentNode.point,newnode)

                if checkCollision(newnode, goalPoint.point, 10):
                    currentState = 'goalFound'

                    goalNode = nodes[len(nodes)-1]

                
            else:
                return

        #handle events  for setting the goal and starting point
        for e in pygame.event.get():

            # For stopping the simulation
            if e.type == QUIT or (e.type == KEYUP and e.key == K_ESCAPE):
                sys.exit("Exiting")

            # For setting the start and goal points
            if e.type == MOUSEBUTTONDOWN:
                if currentState == 'init':

                    # checking if the start position of the robot is set
                    if startPointSet == False:
                        nodes = []
                        if collideObstacle(e.pos) == False:
                            print('initiale point set: '+str(e.pos))

                            startPoint = treeNode(e.pos, None)
                            nodes.append(startPoint) # Start in the center
                            startPointSet = True
                            pygame.draw.circle(screen, red, startPoint.point, 10)
                            
                    # checking if the goal position is set        
                    elif goalPoseSet == False:
                        print('goal point set: '+str(e.pos))
                        if collideObstacle(e.pos) == False:
                            goalPoint = treeNode(e.pos,None)
                            goalPoseSet = True
                            pygame.draw.circle(screen, green, goalPoint.point, 10)
                            currentState = 'buildTree'
                else:
                    currentState = 'init'
                    startPointSet = False
                    goalPoseSet = False
                    reset()

        pygame.display.update()
        fpsClock.tick(10000)

In [43]:
if __name__ == '__main__':
    startRRT()

goal point not yet set
goal point not yet set
goal point not yet set
goal point not yet set
goal point not yet set
goal point not yet set
goal point not yet set
goal point not yet set
goal point not yet set
goal point not yet set
goal point not yet set
goal point not yet set
goal point not yet set
goal point not yet set
goal point not yet set
goal point not yet set
goal point not yet set
goal point not yet set
goal point not yet set
goal point not yet set
goal point not yet set
goal point not yet set
goal point not yet set
goal point not yet set
goal point not yet set
goal point not yet set
initiale point set: (23, 25)
goal point not yet set
goal point not yet set
goal point not yet set
goal point not yet set
goal point not yet set
goal point not yet set
goal point not yet set
goal point not yet set
goal point not yet set
goal point not yet set
goal point not yet set
goal point not yet set
goal point not yet set
goal point not yet set
goal point not yet set
goal point not yet set
goal 

SystemExit: Exiting